In [71]:
import os
import urllib
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [20]:
path = r'/home/owner/CoAgMET/Relative_humidity'
os.chdir(path)

fmcsv = (path + r'\fm_csv.csv')
dycsv = (path + r'\dy_csv.csv')

f = open(fmcsv,'w+')
d = open(dycsv,'w+')

In [41]:
urllib.request.urlretrieve(
    'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=five_minute' + 
    '&sids=rfd01&sdate=1992-06-04&edate=2018-12-31' +
    '&elems=tmean,rh,vp,sr,ws,wind_vec,pp,st5,st15,gust,gustdir',
    filename=fmcsv)
fmdf = pd.read_csv(fmcsv)

In [42]:
fmdf = fmdf.reset_index()
fmdf.columns = ['station','datetime','tmean','rh','vp','sr','ws',
                'wind_vec','pp','st5','st15','gust','gustdir']

In [43]:
fmdf['datetime'] = pd.to_datetime(fmdf.datetime,errors='coerce')

idx_max = fmdf.groupby(fmdf.datetime.dt.date,sort=False)['rh'].idxmax()
idx_min = fmdf.groupby(fmdf.datetime.dt.date,sort=False)['rh'].idxmin()

fmdf_max = fmdf.loc[idx_max,]
fmdf_min = fmdf.loc[idx_min,]

In [44]:
stdev = fmdf.groupby(fmdf.datetime.dt.date,sort=False).std()

stdev = stdev.reset_index()

stdev['datetime'] = pd.to_datetime(stdev.datetime, errors='coerce')

stdev.columns = ['datetime','tmean_stdev','rh_stdev','vp_stdev','sr_stdev',
                 'ws_stdev','wind_vec_stdev','pp','st5_stdev',
                 'st15_stdev','gust_stdev','gustdir_stdev']

stdev = stdev.drop('pp',axis=1)

In [45]:
fmdf_max['datetime'] = pd.to_datetime(fmdf_max.datetime, errors='coerce')
fmdf_min['datetime'] = pd.to_datetime(fmdf_min.datetime, errors='coerce')

fmdf_max['date'] = fmdf_max.datetime.dt.date
fmdf_min['date'] = fmdf_min.datetime.dt.date

In [47]:
fmdf_max.columns = ['station','datetime_max','tmean_max','rh_max','vp_max','sr_max',
                   'ws_max','wind_vec_max','pp','st5_max','st15_max','gust_max',
                   'gustdir_max','date']

fmdf_max = fmdf_max.drop('pp',axis=1)

In [48]:
fmdf_min.columns = ['station','datetime_min','tmean_min','rh_min','vp_min','sr_min',
                   'ws_min','wind_vec_min','pp','st5_min','st15_min','gust_min',
                   'gustdir_min','date']

fmdf_min = fmdf_min.drop('pp',axis=1)

In [49]:
fmdf_mxmn = fmdf_max.merge(fmdf_min, on=fmdf_max.date)

fmdf_mxmn.columns.values[0] = 'date'
fmdf_mxmn.columns.values[1] = 'station'

stuff = ['date_x','station_y','date_y']

fmdf_mxmn = fmdf_mxmn.drop(stuff,axis=1)

In [51]:
first_obs = fmdf_mxmn.date[0]
str(first_obs)

'2016-12-13'

In [60]:
urllib.request.urlretrieve(
    'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily' +
    '&sids=rfd01&sdate=' + str(first_obs) + '&edate=2018-12-31&elems=' +
    'rhmax,rhmxtm,rhmin,rhmntm',filename=dycsv)

dydf = pd.read_csv(dycsv)
dydf = dydf.reset_index()
dydf.columns = ['station','date','rhmax','rhmax_time','rhmin','rhmin_time']

In [61]:
dydf = dydf.drop(dydf.tail(1).index)

In [62]:
dydf['rhmxtm'] = dydf['date'] + ' ' + dydf['rhmax_time']
dydf['rhmxtm'] = pd.to_datetime(dydf['rhmxtm'],format='%Y/%m/%d %H:%M:%S')

dydf['rhmntm'] = dydf['date'] + ' ' + dydf['rhmin_time']
dydf['rhmntm'] = pd.to_datetime(dydf['rhmntm'],format='%Y/%m/%d %H:%M:%S')

dydf['date'] = pd.to_datetime(dydf.date, errors='coerce')

In [66]:
df = fmdf_mxmn.merge(dydf,on=fmdf_mxmn.date)

In [68]:
list(df.columns)
df.columns.values[0] = 'date'
df.columns.values[2] = 'station'

stuff = ['station_y','date_y','date_x']
df = df.drop(stuff, axis=1)

In [69]:
df

,date,station,datetime_max,tmean_max,rh_max,vp_max,sr_max,ws_max,wind_vec_max,st5_max,...,st5_min,st15_min,gust_min,gustdir_min,rhmax,rhmax_time,rhmin,rhmin_time,rhmxtm,rhmntm
0,2016-12-13,rfd01,2016-12-13 23:55:00,-8.830,0.878,0.276,0.000,0.320,132.50,-0.144,...,-0.115,1.196,2.225,12.67,0.888,23:53:30,0.445,15:13:30,2016-12-13 23:53:30,2016-12-13 15:13:30
1,2016-12-14,rfd01,2016-12-14 03:20:00,-10.520,0.939,0.258,0.000,0.456,9.06,-0.435,...,-0.266,0.948,4.400,112.30,0.945,05:06:50,0.391,15:06:50,2016-12-14 05:06:50,2016-12-14 15:06:50
2,2016-12-15,rfd01,2016-12-15 04:35:00,-11.860,0.844,0.209,0.000,1.012,284.00,-1.463,...,-0.390,0.536,2.750,334.70,0.849,05:47:30,0.372,15:22:50,2016-12-15 05:47:30,2016-12-15 15:22:50
3,2016-12-16,rfd01,2016-12-16 21:30:00,-6.440,0.783,0.296,0.000,3.643,44.61,0.481,...,2.677,1.259,7.325,293.30,0.788,21:27:30,0.200,15:23:00,2016-12-16 21:27:30,2016-12-16 15:23:00
4,2016-12-17,rfd01,2016-12-17 14:45:00,-11.580,0.912,0.230,17.100,0.033,70.73,-0.194,...,0.032,1.554,6.350,50.03,0.925,14:41:30,0.623,02:16:50,2016-12-17 14:41:30,2016-12-17 02:16:50
5,2016-12-18,rfd01,2016-12-18 19:50:00,-22.800,0.888,0.087,0.000,0.975,270.30,-0.396,...,-0.333,0.902,2.150,210.10,0.899,19:46:30,0.637,14:27:40,2016-12-18 19:46:30,2016-12-18 14:27:40
6,2016-12-19,rfd01,2016-12-19 22:45:00,-11.850,0.946,0.234,0.000,1.158,302.50,-0.594,...,-0.866,0.659,0.875,46.21,0.965,21:57:10,0.519,14:51:10,2016-12-19 21:57:10,2016-12-19 14:51:10
7,2016-12-20,rfd01,2016-12-20 23:25:00,-6.781,0.980,0.361,0.000,0.978,219.60,-0.170,...,-0.348,0.679,0.875,185.00,0.984,23:21:30,0.311,14:29:20,2016-12-20 23:21:30,2016-12-20 14:29:20
8,2016-12-21,rfd01,2016-12-21 21:25:00,-4.900,0.986,0.419,0.000,0.000,0.00,-0.041,...,-0.226,0.901,2.525,161.10,0.989,21:24:40,0.377,10:59:10,2016-12-21 21:24:40,2016-12-21 10:59:10
9,2016-12-22,rfd01,2016-12-22 12:45:00,-0.071,1.011,0.614,20.800,3.973,115.80,-0.044,...,-0.007,1.063,1.475,102.10,1.012,12:44:50,0.796,15:57:40,2016-12-22 12:44:50,2016-12-22 15:57:40


In [73]:
df['rhmax_diff'] = abs(df.rh_max - df.rhmax)
df['rhmin_diff'] = abs(df.rh_min - df.rhmin)

df['rhmxtm_diff'] = abs(df.datetime_max.values - df.rhmxtm.values)
df['rhmxtm_diff'] = df['rhmxtm_diff'] / np.timedelta64(1,'m')

df['rhmntm_diff'] = abs(df.datetime_min.values - df.rhmntm.values)
df['rhmntm_diff'] = df['rhmntm_diff'] / np.timedelta64(1,'m')

In [74]:
summary = df.describe()
summary

,tmean_max,rh_max,vp_max,sr_max,ws_max,wind_vec_max,st5_max,st15_max,gust_max,gustdir_max,...,st5_min,st15_min,gust_min,gustdir_min,rhmax,rhmin,rhmax_diff,rhmin_diff,rhmxtm_diff,rhmntm_diff
count,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,748.000000,...,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000
mean,3.934009,0.881567,0.857244,0.760677,1.045202,176.857713,11.488377,13.426597,1.652844,206.766979,...,17.055653,14.313287,4.598131,149.651335,0.883192,0.258182,0.013299,0.015138,46.052960,25.677125
std,9.835219,0.131782,0.542611,2.182054,1.272226,120.326706,9.235922,9.120970,1.658638,98.269567,...,11.324238,9.645004,2.764295,93.257259,0.137883,0.177416,0.048826,0.014820,199.299904,82.240711
min,-22.800000,0.364000,0.087000,0.000000,0.000000,0.000000,-5.065000,-1.805000,0.000000,0.000000,...,-1.493000,-1.330000,0.000000,0.024000,0.130000,0.033000,0.000000,0.001000,0.000000,0.000000
25%,-4.537000,0.843000,0.382000,0.000000,0.122000,54.520000,2.482000,4.679000,0.800000,141.400000,...,6.852000,5.099000,2.525000,80.200000,0.848000,0.127000,0.002000,0.006000,1.500000,2.166667
50%,3.448000,0.925000,0.688000,0.000000,0.683000,199.000000,10.140000,12.240000,1.250000,230.400000,...,15.940000,13.140000,4.025000,126.900000,0.931000,0.207000,0.005000,0.012000,3.000000,4.500000
75%,13.360000,0.972000,1.311000,0.363000,1.358000,282.600000,20.940000,22.950000,2.000000,285.525000,...,28.020000,24.300000,6.050000,226.600000,0.975000,0.332000,0.011000,0.021000,5.833333,16.000000
max,23.970000,1.039000,2.261000,24.650000,8.630000,359.300000,30.560000,30.810000,10.620000,353.900000,...,38.850000,31.530000,18.580000,354.500000,1.037000,0.992000,0.612000,0.287000,1435.000000,1383.500000


In [76]:
df

,date,station,datetime_max,tmean_max,rh_max,vp_max,sr_max,ws_max,wind_vec_max,st5_max,...,rhmax,rhmax_time,rhmin,rhmin_time,rhmxtm,rhmntm,rhmax_diff,rhmin_diff,rhmxtm_diff,rhmntm_diff
0,2016-12-13,rfd01,2016-12-13 23:55:00,-8.830,0.878,0.276,0.000,0.320,132.50,-0.144,...,0.888,23:53:30,0.445,15:13:30,2016-12-13 23:53:30,2016-12-13 15:13:30,0.010,0.004,1.500000,1.500000
1,2016-12-14,rfd01,2016-12-14 03:20:00,-10.520,0.939,0.258,0.000,0.456,9.06,-0.435,...,0.945,05:06:50,0.391,15:06:50,2016-12-14 05:06:50,2016-12-14 15:06:50,0.006,0.007,106.833333,1.833333
2,2016-12-15,rfd01,2016-12-15 04:35:00,-11.860,0.844,0.209,0.000,1.012,284.00,-1.463,...,0.849,05:47:30,0.372,15:22:50,2016-12-15 05:47:30,2016-12-15 15:22:50,0.005,0.008,72.500000,2.166667
3,2016-12-16,rfd01,2016-12-16 21:30:00,-6.440,0.783,0.296,0.000,3.643,44.61,0.481,...,0.788,21:27:30,0.200,15:23:00,2016-12-16 21:27:30,2016-12-16 15:23:00,0.005,0.019,2.500000,2.000000
4,2016-12-17,rfd01,2016-12-17 14:45:00,-11.580,0.912,0.230,17.100,0.033,70.73,-0.194,...,0.925,14:41:30,0.623,02:16:50,2016-12-17 14:41:30,2016-12-17 02:16:50,0.013,0.005,3.500000,3.166667
5,2016-12-18,rfd01,2016-12-18 19:50:00,-22.800,0.888,0.087,0.000,0.975,270.30,-0.396,...,0.899,19:46:30,0.637,14:27:40,2016-12-18 19:46:30,2016-12-18 14:27:40,0.011,0.019,3.500000,2.666667
6,2016-12-19,rfd01,2016-12-19 22:45:00,-11.850,0.946,0.234,0.000,1.158,302.50,-0.594,...,0.965,21:57:10,0.519,14:51:10,2016-12-19 21:57:10,2016-12-19 14:51:10,0.019,0.015,47.833333,251.166667
7,2016-12-20,rfd01,2016-12-20 23:25:00,-6.781,0.980,0.361,0.000,0.978,219.60,-0.170,...,0.984,23:21:30,0.311,14:29:20,2016-12-20 23:21:30,2016-12-20 14:29:20,0.004,0.014,3.500000,0.666667
8,2016-12-21,rfd01,2016-12-21 21:25:00,-4.900,0.986,0.419,0.000,0.000,0.00,-0.041,...,0.989,21:24:40,0.377,10:59:10,2016-12-21 21:24:40,2016-12-21 10:59:10,0.003,0.027,0.333333,0.833333
9,2016-12-22,rfd01,2016-12-22 12:45:00,-0.071,1.011,0.614,20.800,3.973,115.80,-0.044,...,1.012,12:44:50,0.796,15:57:40,2016-12-22 12:44:50,2016-12-22 15:57:40,0.001,0.005,0.166667,2.333333


In [79]:
max_fig = go.Figure(data=[go.Scatter(
    x=df.rhmax,y=df.rh_max,text=df.date,mode='markers',name='rhmax')],
                   layout=go.Layout(title=go.layout.Title(text='5 Minute vs Daily Rhmax'),
                                   xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Daily Rhmax')),
                                   yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='5-Minute Rhmax'))))
max_fig.show()

In [80]:
min_fig = go.Figure(data=[go.Scatter(
    x=df.rhmin,y=df.rh_min,text=df.date,mode='markers',name='rhmin')],
                   layout=go.Layout(title=go.layout.Title(text='5 Minute vs Daily Rhmin'),
                                   xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Daily Rhmin')),
                                   yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='5-Minute Rhmin'))))
min_fig.show()

In [81]:
rhmxtm_fig = go.Figure(data=[go.Scatter(
    x=df.rhmax,y=df.rhmxtm_diff,text=df.date,mode='markers',name='rhmxtm')],
                   layout=go.Layout(title=go.layout.Title(text='Rhmax vs Time Differece'),
                                   xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Daily Rhmax')),
                                   yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Rhmax Time Difference'))))
rhmxtm_fig.show()

In [82]:
rhmntm_fig = go.Figure(data=[go.Scatter(
    x=df.rhmin,y=df.rhmntm_diff,text=df.date,mode='markers',name='rhmin')],
                   layout=go.Layout(title=go.layout.Title(text='Rhmin vs Time Difference'),
                                   xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Daily Rhmin')),
                                   yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Rhmin Time Difference'))))
rhmntm_fig.show()

In [83]:
time_fig = go.Figure(data=[go.Scatter(
    x=df.rhmntm_diff,y=df.rhmxtm_diff,text=df.date,mode='markers',name='rhmin')],
                   layout=go.Layout(title=go.layout.Title(text='Max Time Diff v Min Time Diff'),
                                   xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Daily Rhmin Time-diff')),
                                   yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Daily Rhmax Time-diff'))))
time_fig.show()